In [6]:
import pandas as pd
import numpy as np
import os
import random

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit import DataStructs
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf

In [ ]:
# GPU 설정
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [ ]:
CFG = {
    'NBITS': 2048,
    'SEED': 42,
}

In [9]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

seed_everything(CFG['SEED'])  # Seed 고정

In [10]:
def smiles_to_fingerprint(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is not None:
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=CFG['NBITS'])
        return np.array(fp)
    else:
        return np.zeros((CFG['NBITS'],), dtype=int)

In [7]:
# 학습 ChEMBL 데이터 로드
chembl_data = pd.read_csv('./data2/train.csv')  # 예시 파일 이름
chembl_data.head()

,Molecule ChEMBL ID,Standard Type,Standard Relation,Standard Value,Standard Units,pChEMBL Value,Assay ChEMBL ID,Target ChEMBL ID,Target Name,Target Organism,Target Type,Document ChEMBL ID,IC50_nM,pIC50,Smiles
0,CHEMBL4443947,IC50,'=',0.022,nM,10.66,CHEMBL4361896,CHEMBL3778,Interleukin-1 receptor-associated kinase 4,Homo sapiens,SINGLE PROTEIN,CHEMBL4359855,0.022,10.66,CN[C@@H](C)C(=O)N[C@H](C(=O)N1C[C@@H](NC(=O)CC...
1,CHEMBL4556091,IC50,'=',0.026,nM,10.59,CHEMBL4345131,CHEMBL3778,Interleukin-1 receptor-associated kinase 4,Homo sapiens,SINGLE PROTEIN,CHEMBL4342485,0.026,10.59,CC(C)(O)[C@H](F)CN1Cc2cc(NC(=O)c3cnn4cccnc34)c...
2,CHEMBL4566431,IC50,'=',0.078,nM,10.11,CHEMBL4345131,CHEMBL3778,Interleukin-1 receptor-associated kinase 4,Homo sapiens,SINGLE PROTEIN,CHEMBL4342485,0.078,10.11,CC(C)(O)[C@H](F)CN1Cc2cc(NC(=O)c3cnn4cccnc34)c...
3,CHEMBL4545898,IC50,'=',0.081,nM,10.09,CHEMBL4345131,CHEMBL3778,Interleukin-1 receptor-associated kinase 4,Homo sapiens,SINGLE PROTEIN,CHEMBL4342485,0.081,10.09,CC(C)(O)[C@H](F)CN1Cc2cc(NC(=O)c3cnn4cccnc34)c...
4,CHEMBL4448950,IC50,'=',0.099,nM,10.00,CHEMBL4361896,CHEMBL3778,Interleukin-1 receptor-associated kinase 4,Homo sapiens,SINGLE PROTEIN,CHEMBL4359855,0.099,10.00,COc1cc2c(OC[C@@H]3CCC(=O)N3)ncc(C#CCCCCCCCCCCC...


In [14]:
train = chembl_data[['Smiles', 'pIC50']].copy()  # .copy()를 사용하여 SettingWithCopyWarning 방지
train['Fingerprint'] = train['Smiles'].apply(smiles_to_fingerprint)

train_x = np.stack(train['Fingerprint'].values)
train_y = train['pIC50'].values


[13:39:52] DEPRECATION WARNING: please use MorganGenerator
[13:39:52] DEPRECATION WARNING: please use MorganGenerator
[13:39:52] DEPRECATION WARNING: please use MorganGenerator
[13:39:52] DEPRECATION WARNING: please use MorganGenerator
[13:39:52] DEPRECATION WARNING: please use MorganGenerator
[13:39:52] DEPRECATION WARNING: please use MorganGenerator
[13:39:52] DEPRECATION WARNING: please use MorganGenerator
[13:39:52] DEPRECATION WARNING: please use MorganGenerator
[13:39:52] DEPRECATION WARNING: please use MorganGenerator
[13:39:52] DEPRECATION WARNING: please use MorganGenerator
[13:39:52] DEPRECATION WARNING: please use MorganGenerator
[13:39:52] DEPRECATION WARNING: please use MorganGenerator
[13:39:52] DEPRECATION WARNING: please use MorganGenerator
[13:39:52] DEPRECATION WARNING: please use MorganGenerator
[13:39:52] DEPRECATION WARNING: please use MorganGenerator
[13:39:52] DEPRECATION WARNING: please use MorganGenerator
[13:39:52] DEPRECATION WARNING: please use MorganGenerat

In [29]:

# 학습 및 검증 데이터 분리
#train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.3, random_state=CFG['SEED'])

# 특징 스케일링
#scaler = StandardScaler()
#train_x_scaled = scaler.fit_transform(train_x)



In [20]:
# Define Transformer Model
def build_transformer_model(input_shape, head_size, num_heads, ff_dim, dropout=0):
    inputs = tf.keras.Input(shape=(input_shape,))
    x = tf.keras.layers.Reshape((1, input_shape))(inputs)  # Reshape for transformer input
    x = tf.keras.layers.Dense(ff_dim, activation="relu")(x)
    x = tf.keras.layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = tf.keras.layers.Dropout(dropout)(x)
    x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x)
    x = tf.keras.layers.GlobalAveragePooling1D()(x)
    x = tf.keras.layers.Dense(20, activation="relu")(x)
    x = tf.keras.layers.Dense(1)(x)
    model = tf.keras.Model(inputs, x)
    return model


In [30]:

input_shape = train_x.shape[1]
model = build_transformer_model(input_shape, head_size=64, num_heads=4, ff_dim=64, dropout=0.1)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss='mse')

# Train the model
history = model.fit(
    train_x, train_y,
    validation_split=0.3,
    epochs=300,
    batch_size=32
)


Epoch 1/300
21/21 [==============================] - 1s 23ms/step - loss: 69.6374 - val_loss: 59.9991
Epoch 2/300
21/21 [==============================] - 0s 13ms/step - loss: 54.9938 - val_loss: 44.4998
Epoch 3/300
21/21 [==============================] - 0s 12ms/step - loss: 41.7212 - val_loss: 31.6050
Epoch 4/300
21/21 [==============================] - 0s 12ms/step - loss: 30.3832 - val_loss: 21.7539
Epoch 5/300
21/21 [==============================] - 0s 13ms/step - loss: 22.6819 - val_loss: 14.7258
Epoch 6/300
21/21 [==============================] - 0s 14ms/step - loss: 16.3408 - val_loss: 9.7998
Epoch 7/300
21/21 [==============================] - 0s 13ms/step - loss: 11.8979 - val_loss: 6.4694
Epoch 8/300
21/21 [==============================] - 0s 12ms/step - loss: 8.5080 - val_loss: 4.2226
Epoch 9/300
21/21 [==============================] - 0s 13ms/step - loss: 6.0220 - val_loss: 2.7520
Epoch 10/300
21/21 [==============================] - 0s 13ms/step - loss: 4.7422 - val_

In [25]:

# Validation 데이터로부터의 학습 모델 평가
#val_x_scaled = scaler.transform(val_x)
#val_y_pred = model.predict(val_x_scaled)
mse = mean_squared_error(val_y, val_y_pred)
rmse = np.sqrt(mse)
print(f'RMSE: {rmse}')

RMSE: 0.9544668246925485


In [31]:

def pIC50_to_IC50(pic50_values):
    """Convert pIC50 values to IC50 (nM)."""
    return 10 ** (9 - pic50_values)

In [32]:
# Make predictions on test data
test = pd.read_csv('./data2/test.csv')
test['Fingerprint'] = test['Smiles'].apply(smiles_to_fingerprint)
test_x = np.stack(test['Fingerprint'].values)
test_y_pred = model.predict(test_x)
#test_x_scaled = scaler.transform(test_x)
#test_y_pred = model.predict(test_x_scaled)


[13:57:01] DEPRECATION WARNING: please use MorganGenerator
[13:57:01] DEPRECATION WARNING: please use MorganGenerator
[13:57:01] DEPRECATION WARNING: please use MorganGenerator
[13:57:01] DEPRECATION WARNING: please use MorganGenerator
[13:57:01] DEPRECATION WARNING: please use MorganGenerator
[13:57:01] DEPRECATION WARNING: please use MorganGenerator
[13:57:01] DEPRECATION WARNING: please use MorganGenerator
[13:57:01] DEPRECATION WARNING: please use MorganGenerator
[13:57:01] DEPRECATION WARNING: please use MorganGenerator
[13:57:01] DEPRECATION WARNING: please use MorganGenerator
[13:57:01] DEPRECATION WARNING: please use MorganGenerator
[13:57:01] DEPRECATION WARNING: please use MorganGenerator
[13:57:01] DEPRECATION WARNING: please use MorganGenerator
[13:57:01] DEPRECATION WARNING: please use MorganGenerator
[13:57:01] DEPRECATION WARNING: please use MorganGenerator
[13:57:01] DEPRECATION WARNING: please use MorganGenerator
[13:57:01] DEPRECATION WARNING: please use MorganGenerat

In [33]:

# Prepare submission
submit = pd.read_csv('./data2/sample_submission.csv')
#submit['IC50_nM'] = 10 ** (-test_y_pred)  # Convert pIC50 to IC50
submit['IC50_nM'] = pIC50_to_IC50(test_y_pred)
submit.to_csv('./data2/transformer_submit_full_train_no_transform.csv', index=False)